## HBAT2 Analysis with 3D Visualization

This notebook demonstrates how to use HBAT to analyze molecular interactions in protein structures and visualize them using py3Dmol.

We'll analyze the PDB structure 6RSA (Ribonuclease A) and visualize different types of interactions.

## Installation

If you haven't installed the required packages, uncomment and run:

In [ ]:
# !pip install hbat

## Import Libraries

In [ ]:
import hbat
from hbat.core.analysis import NPMolecularInteractionAnalyzer, AnalysisParameters
import pandas as pd
from pathlib import Path

print(f"HBAT version: {hbat.__version__}")

## Load and Analyze PDB Structure

We'll use the 6RSA structure from the example_pdb_files directory.

In [ ]:
# Path to PDB file
pdb_file = "../example_pdb_files/6rsa.pdb"

# Create analyzer with default parameters
parameters = AnalysisParameters()
analyzer = NPMolecularInteractionAnalyzer(parameters)

# Run analysis
print("Analyzing structure...")
success = analyzer.analyze_file(pdb_file)

if success:
    print("✓ Analysis completed successfully")
else:
    print("✗ Analysis failed")

## Summary Statistics

In [ ]:
summary = analyzer.get_summary()

print("=" * 60)
print(f"Structure: {pdb_file}")
print("=" * 60)
print(f"Hydrogen Bonds:        {summary['hydrogen_bonds']['count']:4d}")
print(f"Halogen Bonds:         {summary['halogen_bonds']['count']:4d}")
print(f"π Interactions:        {summary['pi_interactions']['count']:4d}")
print(f"π-π Stacking:          {summary.get('pi_pi_interactions', {}).get('count', 0):4d}")
print(f"Carbonyl n→π*:         {summary.get('carbonyl_interactions', {}).get('count', 0):4d}")
print(f"n→π* Interactions:     {summary.get('n_pi_interactions', {}).get('count', 0):4d}")
print(f"Cooperativity Chains:  {summary['cooperativity_chains']['count']:4d}")
print("=" * 60)

## Convert Interactions to DataFrames

In [ ]:
# Hydrogen bonds to DataFrame
hbonds_data = []
for hb in analyzer.hydrogen_bonds:
    hbonds_data.append({
        'Donor': f"{hb.donor.res_name}{hb.donor.res_seq}:{hb.donor.name}",
        'Acceptor': f"{hb.acceptor.res_name}{hb.acceptor.res_seq}:{hb.acceptor.name}",
        'Distance (Å)': round(hb.distance, 2),
        'Angle (°)': round(hb.angle, 1),
        'Type': hb.bond_type
    })

df_hbonds = pd.DataFrame(hbonds_data)
print(f"\nFirst 10 Hydrogen Bonds:")
display(df_hbonds.head(10))

## 3D Visualization with py3Dmol

Let's visualize the protein structure with different interaction types highlighted.

In [ ]:
# Use HBAT's visualization function to show structure with interactions
from hbat.visualization import visualize_structure_with_interactions

viewer = visualize_structure_with_interactions(
    pdb_file,
    analyzer,
    width=800,
    height=600,
    max_hbonds=20,
    max_pi_interactions=10,
    show_all_halogen_bonds=True,
    show_water=True
)

viewer.show()

## Focused View: Specific Residue Interactions

Let's zoom in on a specific region to see interactions in detail.

In [ ]:
# Use HBAT's visualization function to focus on a specific residue
from hbat.visualization import visualize_residue_interactions

# Target residue to visualize
target_res_num = 50

# Find interactions involving this residue and extract its info
residue_hbonds = [
    hb for hb in analyzer.hydrogen_bonds 
    if hb.donor.res_seq == target_res_num or hb.acceptor.res_seq == target_res_num
]

if residue_hbonds:
    # Get chain and residue name from the first matching bond
    hb = residue_hbonds[0]
    if hb.donor.res_seq == target_res_num:
        chain = hb.donor.chain_id
        residue_name = hb.donor.res_name
    else:
        chain = hb.acceptor.chain_id
        residue_name = hb.acceptor.res_name
    
    print(f"Found {len(residue_hbonds)} hydrogen bonds involving residue {chain}{target_res_num}{residue_name}")
    
    viewer2 = visualize_residue_interactions(
        pdb_file,
        analyzer,
        chain=chain,
        residue_number=target_res_num,
        residue_name=residue_name,
        width=800,
        height=600,
        show_labels=True,
        show_water=True
    )
    
    viewer2.show()
else:
    print(f"No hydrogen bonds found for residue {target_res_num}")

## Export Interaction Data

In [ ]:
# Export to JSON for further analysis
from hbat.export.results import export_to_json_single_file

output_file = "6rsa_analysis_results.json"
export_to_json_single_file(analyzer, output_file, pdb_file)
print(f"Results exported to {output_file}")

## Interaction Statistics

In [ ]:
# Analyze hydrogen bond types
if len(analyzer.hydrogen_bonds) > 0:
    bond_types = {}
    for hb in analyzer.hydrogen_bonds:
        bond_type = hb.bond_type
        bond_types[bond_type] = bond_types.get(bond_type, 0) + 1
    
    print("\nHydrogen Bond Types:")
    print("-" * 40)
    for bond_type, count in sorted(bond_types.items(), key=lambda x: x[1], reverse=True):
        print(f"{bond_type:20s}: {count:3d}")

# Distance distribution
distances = [hb.distance for hb in analyzer.hydrogen_bonds]
if distances:
    print(f"\nHydrogen Bond Distance Statistics:")
    print("-" * 40)
    print(f"Mean:   {sum(distances)/len(distances):.2f} Å")
    print(f"Min:    {min(distances):.2f} Å")
    print(f"Max:    {max(distances):.2f} Å")

## Cooperativity Chains

Visualize cooperativity chains (sequences of connected hydrogen bonds).

In [ ]:
chains = analyzer.cooperativity_chains

if chains:
    print(f"Found {len(chains)} cooperativity chains")
    print(f"\nLongest chains:")
    
    # Sort by chain_length
    sorted_chains = sorted(chains, key=lambda c: c.chain_length, reverse=True)
    
    for i, chain in enumerate(sorted_chains[:5]):
        print(f"\nChain {i+1}: {chain.chain_length} bonds")
        for j, hb in enumerate(chain.interactions):
            print(f"  {j+1}. {hb.donor.res_name}{hb.donor.res_seq}:{hb.donor.name} → "
                  f"{hb.acceptor.res_name}{hb.acceptor.res_seq}:{hb.acceptor.name} "
                  f"({hb.distance:.2f} Å)")
else:
    print("No cooperativity chains found")

## Visualize Cooperativity Chains with Graphviz

Create network diagrams showing the flow of interactions in cooperativity chains.

In [ ]:
# Visualize cooperativity chains using HBAT's visualization functions
from hbat.visualization import create_chain_graph, render_chain_graphviz

if chains and len(chains) > 0:
    # Get the longest chain
    longest_chain = sorted(chains, key=lambda c: c.chain_length, reverse=True)[0]
    print(f"Visualizing longest cooperativity chain ({longest_chain.chain_length} interactions)")
    
    try:
        # Use HBAT's built-in visualization function (reuses GUI code)
        dot = render_chain_graphviz(
            longest_chain,
            engine='dot',         # Layout engine (dot, neato, fdp, circo, twopi)
            rankdir='LR',         # Left-to-right layout
            filename='cooperativity_chain',  # Save to file
            format='png'          # Output format
        )
        
        # Display in Jupyter (graphviz objects render automatically)
        display(dot)
        
        print("\nGraph saved to cooperativity_chain.png")
        print("\nNote: This uses the same visualization logic as HBAT's GUI")
        
    except ImportError as e:
        print(f"Graphviz not available: {e}")
        print("\nInstall graphviz:")
        print("  pip install graphviz")
        print("\nAlso install Graphviz system software:")
        print("  - Ubuntu/Debian: sudo apt-get install graphviz")
        print("  - macOS: brew install graphviz")
        print("  - Windows: Download from https://graphviz.org/download/")
        
        # Fallback: create NetworkX graph to show structure
        print("\nFallback: Creating NetworkX graph...")
        G = create_chain_graph(longest_chain)
        print(f"Graph has {len(G.nodes())} nodes and {len(G.edges())} edges")
        print(f"Nodes: {list(G.nodes())}")
        print(f"\nEdges:")
        for u, v in G.edges():
            print(f"  {u} -> {v}")
else:
    print("No cooperativity chains to visualize")

## Custom Analysis with Different Parameters

Run analysis with custom parameter settings.

In [ ]:
# Create custom parameters for strict hydrogen bonds
strict_params = AnalysisParameters(
    hb_distance_cutoff=2.2,  # Stricter distance
    hb_angle_cutoff=130.0,   # Stricter angle
    analysis_mode='complete'
)

# Run analysis with strict parameters
strict_analyzer = NPMolecularInteractionAnalyzer(strict_params)
strict_analyzer.analyze_file(pdb_file)

print("Comparison: Default vs Strict Parameters")
print("="*60)
print(f"{'Interaction Type':<30} {'Default':<15} {'Strict':<15}")
print("-"*60)
print(f"{'Hydrogen Bonds':<30} {len(analyzer.hydrogen_bonds):<15} {len(strict_analyzer.hydrogen_bonds):<15}")
print(f"{'Halogen Bonds':<30} {len(analyzer.halogen_bonds):<15} {len(strict_analyzer.halogen_bonds):<15}")
print(f"{'π Interactions':<30} {len(analyzer.pi_interactions):<15} {len(strict_analyzer.pi_interactions):<15}")

## Summary

This notebook demonstrated:

1. ✓ Loading and analyzing PDB structures with HBAT
2. ✓ Extracting interaction statistics and summaries
3. ✓ 3D visualization of interactions using py3Dmol
4. ✓ Focused visualization of specific residue interactions
5. ✓ Analyzing cooperativity chains
6. ✓ Network visualization of cooperativity chains with Graphviz
7. ✓ Comparing different parameter settings
8. ✓ Exporting results for further analysis

For more information, visit the [HBAT documentation](https://hbat.abhishek-tiwari.com/).